# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [5]:
# Import Python packages 
from cassandra.cluster import Cluster
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import sys
!{sys.executable} -m pip install PrettyTable
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [6]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [7]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [8]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating the Cassandra Cluster

In [9]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Creating the Keyspace

In [10]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
        {'class': 'SimpleStrategy', 'replication_factor': 1}
    """
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [11]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### In this query the analysts want to see the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4. First we must create a table to represent this query. 

In [14]:
music_history_query = """
    CREATE TABLE IF NOT EXISTS music_history (
        session_id int,
        item_in_session int,
        artist text,
        song_title text,
        length float,
        PRIMARY KEY(session_id, item_in_session))
"""
try:
    session.execute(music_history_query)
except Exception as e:
    print(e)         

In [15]:
"""
Here we are inserting rows from the CSV which correspond to the data 
we need in this specific table.
"""

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        insert_music_history = """
            INSERT INTO music_history (session_id, item_in_session, artist, song_title, length) VALUES (%s, %s, %s, %s, %s)
        """
        try:
            session.execute(insert_music_history, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]) ))
        except Exception as e:
                print(e)

In [17]:
# Veryifying that the table has been indeed created and it is what the analysts are looking for. 
query1 = """
    SELECT artist,
        song_title,
        length
    FROM music_history
    WHERE session_id = 338
    AND item_in_session = 4
"""
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

q1 = PrettyTable(['Artist', 'Song Title', 'Length'])
for row in rows:
    q1.add_row([row.artist, row.song_title, row.length])
print(q1)

+-----------+---------------------------------+--------------------+
|   Artist  |            Song Title           |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


### 2. The second query the analysts have asked for is provided in this statement: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182. We first need to create a table that corresponds to their request, then veryify the information from our query. 

In [18]:
music_user_info = """
    CREATE TABLE IF NOT EXISTS music_user_info (
        user_id int,
        session_id int,
        item_in_session int,
        artist text,
        song_title text,
        first_name text,
        last_name text,
        PRIMARY KEY((user_id, session_id), item_in_session))
"""
try:
    session.execute(music_user_info)
except Exception as e:
    print(e)    

In [19]:
"""
As with the above query, we are opening the CSV, choosing the columns/rows of data 
we need, and inserting the information properly into our music_user_info table.
"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        insert_music_user = """
            INSERT INTO music_user_info (user_id, session_id, item_in_session, artist, song_title, first_name, last_name) 
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        try:
            session.execute(insert_music_user, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4] ))
        except Exception as e:
                print(e)

In [22]:
"""
Using our query to verify that the information is correct and the 
data has been uploaded properly 
"""

query2 = """
SELECT artist,
    song_title,
    first_name,
    last_name
    FROM music_user_info
    WHERE user_id = 10
    and session_id = 182
"""
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
q2 = PrettyTable(['Artist', 'Song Title', 'First Name', 'Last Name'])
for row in rows:
    q2.add_row([row.artist, row.song_title, row.first_name, row.last_name])
print(q2)

+-------------------+------------------------------------------------------+------------+-----------+
|       Artist      |                      Song Title                      | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


### The last table we need to create is the music_app_history table, which corresponds to this request: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [24]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
music_app_history_query = """
    CREATE TABLE IF NOT EXISTS music_app_history (
        song text,
        user_id int,
        first_name text,
        last_name text,
        PRIMARY KEY (song, user_id)
    )"""
try:
    session.execute(music_app_history_query)
except Exception as e:
    print(e)         

In [26]:
"""
Inserting the correct information via the CSV we were given
"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_music_app = """
            INSERT INTO music_app_history (song, user_id, first_name, last_name) 
            VALUES (%s, %s, %s, %s)
        """
        try:
            session.execute(insert_music_app, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
                print(e)

In [27]:
"""
Verying that the information we inserted into the table is 
correct relative to the request
"""

query3 = """
    SELECT first_name,
        last_name
        FROM music_app_history
        WHERE song = 'All Hands Against His Own'
        ALLOW FILTERING
"""
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
q3 = PrettyTable(['First Name', 'Last Name'])
for row in rows:
    q3.add_row([row.first_name, row.last_name])
print(q3)

+------------+-----------+
| First Name | Last Name |
+------------+-----------+
|    Sara    |  Johnson  |
|   Tegan    |   Levine  |
| Jacqueline |   Lynch   |
+------------+-----------+


### Drop the tables before closing out the sessions

In [17]:
"""
Dropping the tables 
"""
try:
    session.execute("""DROP TABLE sparkify.music_history;""")
    session.execute("""DROP TABLE sparkify.music_user_info;""")
    session.execute("""DROP TABLE sparkify.music_app_history""")
    print('tables dropped')
except Exception as e:
    print(e)

tables dropped


### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()